In [1]:
import numpy as np

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Input
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

2024-03-26 16:58:55.357257: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-26 16:58:55.399407: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
sign_labels = []
with open('./sign_labels.csv', 'r') as file:
    sign_labels = file.read().splitlines()
file.close()

NUM_CLASSES = len(sign_labels)
sign_labels = np.array(sign_labels)
        

In [3]:
class TrainModel:
    def __init__(self, data_set_path, model_save_path) -> None:
        self.data_set_path = data_set_path
        self.model_save_path = model_save_path

        self.model = Sequential([
            Input((21 * 2, )), # TODO: fuse with the following layer as input_shape param
            Dropout(0.2),
            Dense(20, activation='relu'),
            Dropout(0.4),
            Dense(10, activation='relu'),
            Dense(NUM_CLASSES, activation='softmax') # experiment with different activation functions
        ])
    
    def load_data_set(self):
        X_data = np.loadtxt(self.data_set_path, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))
        y_data = np.loadtxt(self.data_set_path, delimiter=',', dtype='int32', usecols=(0))
        
        return train_test_split(X_data, y_data, test_size=0.2, random_state=42)
    
    def save_model(self):
        self.model.save(self.model_save_path)

In [4]:
data_set_path = './data_set.csv'
model_save_path = './models/model.h5'

Model = TrainModel(data_set_path, model_save_path)

X_train, X_test, y_train, y_test = Model.load_data_set()

2024-03-26 16:58:56.806666: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46851 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:3b:00.0, compute capability: 8.6
2024-03-26 16:58:56.807295: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46864 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:af:00.0, compute capability: 8.6


In [5]:
Model.model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 42)                0         
                                                                 
 dense (Dense)               (None, 20)                860       
                                                                 
 dropout_1 (Dropout)         (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 5)                 55        
                                                                 
Total params: 1125 (4.39 KB)
Trainable params: 1125 (4.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
checkpoint = ModelCheckpoint(Model.model_save_path, verbose=1, save_weights_only=False) # what is this ?
early_stopping = EarlyStopping(patience=20, verbose=1) # what is this ?
tensor_board = TensorBoard(log_dir='./logs', histogram_freq=1) # what is this ?

In [7]:
Model.model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', # diff between this and categorical_crossentropy
    metrics=['accuracy'] # diff between this and categorical_accuracy
)

In [8]:
Model.model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=[tensor_board, checkpoint, early_stopping]
)

Epoch 1/1000


2024-03-26 16:58:58.214301: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-03-26 16:58:58.264871: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x72edb09c0b20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-26 16:58:58.264911: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2024-03-26 16:58:58.264924: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA RTX A6000, Compute Capability 8.6
2024-03-26 16:58:58.275393: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-26 16:58:58.368915: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8907
2024-03-26 16:58:58.48391

 1/11 [=>............................] - ETA: 13s - loss: 1.5988 - accuracy: 0.3125
Epoch 1: saving model to ./models/model.h5
11/11 [==============================] - 2s 36ms/step - loss: 1.6066 - accuracy: 0.2375 - val_loss: 1.5908 - val_accuracy: 0.2514
Epoch 2/1000
 1/11 [=>............................] - ETA: 0s - loss: 1.5974 - accuracy: 0.2344
Epoch 2: saving model to ./models/model.h5
11/11 [==============================] - 0s 11ms/step - loss: 1.5832 - accuracy: 0.2489 - val_loss: 1.5569 - val_accuracy: 0.2571
Epoch 3/1000
 1/11 [=>............................] - ETA: 0s - loss: 1.5626 - accuracy: 0.2812

/home/aicore/miniconda3/envs/actionRec/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 3: saving model to ./models/model.h5
11/11 [==============================] - 0s 11ms/step - loss: 1.5604 - accuracy: 0.2775 - val_loss: 1.5300 - val_accuracy: 0.3943
Epoch 4/1000
 1/11 [=>............................] - ETA: 0s - loss: 1.5111 - accuracy: 0.3906
Epoch 4: saving model to ./models/model.h5
11/11 [==============================] - 0s 10ms/step - loss: 1.5274 - accuracy: 0.3104 - val_loss: 1.5023 - val_accuracy: 0.4686
Epoch 5/1000
 1/11 [=>............................] - ETA: 0s - loss: 1.4971 - accuracy: 0.4062
Epoch 5: saving model to ./models/model.h5
11/11 [==============================] - 0s 10ms/step - loss: 1.5238 - accuracy: 0.2804 - val_loss: 1.4724 - val_accuracy: 0.5314
Epoch 6/1000
 1/11 [=>............................] - ETA: 0s - loss: 1.4954 - accuracy: 0.3281
Epoch 6: saving model to ./models/model.h5
11/11 [==============================] - 0s 10ms/step - loss: 1.4902 - accuracy: 0.3276 - val_loss: 1.4411 - val_accuracy: 0.5657
Epoch 7/1000
 1/11 

In [9]:
val_loss, val_acc = Model.model.evaluate(X_test, y_test, batch_size=64)

3/3 [==============================] - 0s 2ms/step - loss: 0.1402 - accuracy: 0.9771


In [10]:
Model.save_model()

In [ ]:
# TODO:
# rewrite the existing class and add new ones
# plot data
# experiment with different models(diff layers, diff activation functions, diff optimizers, diff loss functions, diff metrics, etc)
# plot training and validation loss and accuracy
# make this code better